<a href="https://colab.research.google.com/github/yosshor/jupyter_notebook_projects/blob/master/GPT_Images_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q openai
!pip install -q gradio
!pip install pyTelegramBotAPI
!pip install Google-Images-Search
!pip install SpeechRecognition
!pip install gTTS
!pip3 install pydub
!pip install pocketsphinx


     |████████████████████████████████| 44 kB 1.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 147 kB 9.3 MB/s 
     |████████████████████████████████| 13.8 MB 5.3 MB/s 
     |████████████████████████████████| 278 kB 79.4 MB/s 
     |████████████████████████████████| 56 kB 4.1 MB/s 
     |████████████████████████████████| 2.3 MB 62.1 MB/s 
     |████████████████████████████████| 106 kB 50.4 MB/s 
     |████████████████████████████████| 55 kB 3.9 MB/s 
     |████████████████████████████████| 84 kB 3.8 MB/s 
     |████████████████████████████████| 84 kB 2.9 MB/s 
     |████████████████████████████████| 64 kB 2.1 MB/s 
     |████████████████████████████████| 80 kB 9.1 MB/s 
     |████████████████████████████████| 69 kB 7.1 MB/s 
     |████████████████████████████████| 58 kB 5.2 MB/s 
     |████████████████████████████████| 50 kB 6.5 MB/s 
Looking in indexes: https

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 32.8 MB 249 kB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 96 kB 3.3 MB/s 
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 29.1 MB 93.2 MB/s 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import openai
# import gradio as gr
import telebot
from telebot import types
import time
import requests
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import os
import re
from google_images_search import GoogleImagesSearch
import nltk
import random
import pprint
from nltk import Tree
import speech_recognition as sr
from gtts import gTTS, lang
from os import path
from pydub import AudioSegment  
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [7]:
telid=[]
with open('telegram_test_bot_token.txt','r') as f:
     telid.append(f.read().split('\n'))

In [11]:
test_token = telid[0][0]
openai.api_key = telid[0][1]
api_google_image_key = telid[0][2]
your_project_cx = telid[0][3].split('=')[1]
bot_url = f'https://api.telegram.org/bot{test_token}/'  
url = bot_url + f'getUpdates'
resp = requests.get(url)
message = json.loads(resp.text)['result']
chat_id = message[0]['message']['from']['id'] #= 319184502

In [12]:
def openai_chat(prompt):
    completions = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=1024,
        n=1,
        temperature=0.5,
    )

    message = completions.choices[0].text
    return message.strip()

In [23]:
def parse_the_recipe_name(message):
  sent = str(re.split(":|\.|\n|\!", message)[0])
  matches = []
  for i in sent.split():
    if i[0].isupper():
      regex = "^[A-Z]\w*$" #"^\s*(?:\b[A-Z]+\b[\s]*)+(?:[:-])\s*$" #
      matches.append(re.findall(regex, i))
      print(i)
  flat_list = [item for sublist in matches[1:] for item in sublist]

  recipe_name = ' '.join(map(str,flat_list))
  print(recipe_name)
  return recipe_name

def parse_the_recipe_name_new(message):
  sent = str(re.split(":|\.|\n|\!", message)[0])
  print(sent_parse(sent)[0])
  return sent_parse(sent)[0]


In [14]:

patterns = """
    NP: {<JJ>*<NN*>+}
    {<JJ>*<NN*><CC>*<NN*>+}
    """

NPChunker = nltk.RegexpParser(patterns)

def prepare_text(input):
    sentences = nltk.sent_tokenize(input)
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    sentences = [nltk.pos_tag(sent) for sent in sentences]
    sentences = [NPChunker.parse(sent) for sent in sentences]
    return sentences


def parsed_text_to_NP(sentences):
    nps = []
    for sent in sentences:
        tree = NPChunker.parse(sent)
        for subtree in tree.subtrees():
            if subtree.label() == 'NP':
                t = subtree
                t = ' '.join(word for word, tag in t.leaves())
                nps.append(t)
    return nps


def sent_parse(input):
    sentences = prepare_text(input)
    nps = parsed_text_to_NP(sentences)
    return nps

def find_nps(text):
    prepared = prepare_text(text)
    parsed = parsed_text_to_NP(prepared)
    final = sent_parse(parsed)

In [172]:
print(sent_parse('I ate peanut butter and beef burger and a cup of coffee for breakfast.'))

['peanut butter', 'beef burger', 'cup', 'coffee', 'breakfast']


In [15]:
# root_dir = "Recipes"
def make_dir(folder_name):
  if not os.path.exists(folder_name):
      os.mkdir(folder_name)
      return True
  return False
# make_dir(root_dir)

In [94]:
from numpy.random import seed
from numpy.random import randint
def download_images(recipe_name,path_to_download,chat_id,file_type):
  gis = GoogleImagesSearch(api_google_image_key, your_project_cx)
  _search_params = {
      'q': f'{recipe_name}',
      'num':100,
      'imagesize':'500x500', #large
      'fileType': f'{file_type}' } #jpg|gif|
  # search first, then download and resize afterwards:
  gis.search(search_params=_search_params,  width=500, height=500)
  if file_type =='gif':
    print(f'len of results : {len(gis.results())}')
    seed(1)
    values = randint(0, 100, 10)
    for i in values:
        gis.results()[i].url  # image direct url
        print(gis.results()[i].url) 
        gis.results()[i].referrer_url  # image referrer url (source)   
        print(gis.results()[i].referrer_url) 
        gis.results()[i].download(path_to_download)
        bot.send_document(chat_id,document=gis.results()[i].url)
  else:
    for image in gis.results():
        image.url  # image direct url
        print(image.url) 
        image.referrer_url  # image referrer url (source)   
        print(image.referrer_url) 
        image.download(path_to_download)  # download image
        image.path  # downloaded local file path
        print(f'image path : {image.path}')
        bot.send_photo(chat_id, photo=open(image.path, 'rb'))

  print('all the images downloaded successfully')

In [17]:
def save_result(path,messasge_to_write):
  with open(path, "w") as text_file:
    text_file.write(messasge_to_write)
  print('File saved')
  return

In [18]:
def speech_to_text(filename):
  # initialize the recognizer
  r = sr.Recognizer()
  # open the file
  with sr.AudioFile(filename) as source:
      # listen for the data (load audio to memory)
      audio_data = r.record(source)
      # recognize (convert from speech to text)
      text = r.recognize_google(audio_data)
      print(text)
      return text

In [19]:
def speechToText(file_name):
                                                                        
    src = f"{file_name}"
    dst = f"{file_name}.wav"
    # convert wav to mp3     
    # https://pythonbasics.org/convert-mp3-to-wav/                                                       
    sound = AudioSegment.from_ogg(src)
    sound.export(dst, format="wav")
    # Initialize the recognizer  
    r = sr.Recognizer()  
    AUDIO_FILE = f"{file_name}.wav"
    with sr.AudioFile(AUDIO_FILE) as source:
        audio2 = r.record(source)  # read the entire audio file
        MyText = r.recognize_sphinx(audio2) 
        MyText = MyText.lower() 
        print(MyText)
        return MyText
    return -1

In [ ]:
def replace_for_result(some_str):
  return some_str.replace('.', '\.').replace('=', '\=').replace('{', '\{').replace('}', '\}').replace('(', '\(').replace(')', '\)').replace('-', '\-').replace('`', '').replace('*', '').replace('#', '\#').replace('!', '\!')

root_dir = "/content/Recipes"

bot = telebot.TeleBot(test_token)

@bot.message_handler(content_types=['voice'])
def voice_processing(message):
    file_info = bot.get_file(message.voice.file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    file_name = f'{random.randint(100000,999999)}.ogg'
    with open(file_name, 'wb') as new_file:
        new_file.write(downloaded_file)
    bot.reply_to(message, "File saved")
    print(f'/content/{file_name.split(".")[0]}')
    text_from_voice = speechToText(f'/content/{file_name}') 
    # text_from_voice = speech_to_text('/content/new_file.ogg') 
    bot.reply_to(message, f'You Said : {text_from_voice}')

@bot.message_handler(commands=['gif'])
def voice_processing(message):
   bot.reply_to(message, f'gif : {message.text.split("/gif")[1:]}')
   dir = '/content/new'
   query = ' '.join(message.text.split('/gif')[1:]).lstrip()
   download_images(query,dir,message.chat.id,'gif')


@bot.message_handler(content_types=['text'])
def print_result(message):
   print(message.text)
   resu = str(openai_chat(message.text))
   print(type(resu))
   print(resu)

   recipe_name = parse_the_recipe_name(resu)
   print(recipe_name)
   if len(recipe_name.split()) == 1:
      print(f'{recipe_name} is one word')
      recipe_name += ' Salad'
   path = os.path.join(root_dir, recipe_name)
   recipe_dir = make_dir(os.path.join(root_dir, recipe_name))
   print(recipe_dir)
   print(f"trying to save the result into {path}/{recipe_name}.txt")
   save_result(path+f'/{recipe_name}.txt',resu)
   bot.send_message(message.chat.id,str(replace_for_result(resu)) ,parse_mode="MarkdownV2")
   try:
     download_images(recipe_name,path,message.chat.id,'png')
   except Exception as e:
     bot.reply_to(message.chat.id, f'some error occured : {str(e)}')

   #bot.send_message(message.chat.id,str(replace_for_result(resu)) ,parse_mode="MarkdownV2")

bot.polling()

In [39]:
%rm -rf '/content/Recipes/Caprese Salad'
%rm -rf '/content/Recipes'
%mkdir '/content/Recipes'

In [97]:
%rm -rf '/content/new'
%mkdir '/content/new'